In [1]:
# make SLDB datasets for BSCTRFM from individual time series
# on traffic dataset

In [2]:
import numpy as np
import pandas as pd

In [3]:
import json

In [4]:
import os

In [5]:
import tensorflow as tf

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import joblib

In [7]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [8]:
# the main source is the traffic dataset PEMS-SF from UCI

In [9]:
# it resides in
dataset_path = '/home/developer/gcp/cbidmltsf/datasets/traffic'

In [10]:
os.listdir(dataset_path)

['PEMS-SF', 'separated_raw']

In [11]:
# a SLDB is produced from separated time series (raw or preprocessed)

# SLDB contents are:
# TFRecord files for training
# TFRecord files for evaluation (if eval required)
# time series pickles for testing

In [12]:
# constant values for positional encodings
hours_in_day = 24
days_in_week = 7
# days_in_month = 30
# days_in_year = 365

In [13]:
# a constant to make sin/cos functions from hours_from_start (the 'age' covariate)
# hourly lectures on first day: from 1 to 23
# hourly lectures on remaining 172 days: from 0 to 23, then
total_hours = 23 + 172*24
total_hours

4151

In [14]:
# no dates available for traffic dataset, then work only on values and covariates

In [15]:
# define global dataset intervals (they might not be precise when missing values exist)

# split the time series in seen (train, eval) and unseen (test) data
# according to academic papers:

# 23 hours + 165 days on seen data, 7 days on unseen data 

# seen data:      lecture 0 to lecture 3982, initial 23 + 165*24 = 3983 lectures

# train/eval split is 0.9/0.1, then

# train data:     lecture 0 to lecture 3584, for 3585 lectures
no_lectures_train_dataset = 3585

# eval data:      lecture 3585 to lecture 3982, for 398 lectures
no_lectures_eval_dataset = 398

# unseen data:    lecture 3983 to lecture 4150, 7*24 = 168 lectures


In [16]:
# build sub-series to be persisted as serialized training examples

# dimensionality of the encoder input
m = 168

# dimensionality of the decoder output 
t = 168

In [17]:
# columns to be included in the SLDB

# use 7D encoder (age, hour-day, day-week)

sldb_columns = [
    # no timestamp information for traffic dataset, work only on pos encoding covariates
    # 'date',
    'id',
    'sequential_id',
    # car lane occupancy was recorded as a variable from 0 to 1
    'occupancy_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    # 'sin_day_month',
    # 'cos_day_month',
    # 'sin_day_year',
    # 'cos_day_year'    
]

In [18]:
sldb = {
    'ts': 'PEMS-SF_SEPARATED_FULL',
    'embedding': {
        'hourly': 168
    },
    'tau': {
        'hourly': 1
    },
    'no_targets': 168,
    'BSCTRFM': 1,
    'preprocessed': 0
}

In [19]:
sldb

{'ts': 'PEMS-SF_SEPARATED_FULL',
 'embedding': {'hourly': 168},
 'tau': {'hourly': 1},
 'no_targets': 168,
 'BSCTRFM': 1,
 'preprocessed': 0}

In [20]:
# a string with the basic specifications of the SLDB, as part of the SLDB identifier

# add the suffix '11D', '07D', etc., to differentiate this SLDB from the original one, which is 09D

# add the suffix MMX to indicate the scaler used was MinMax
# add the suffix STD to indicate the scaler used was Standard

sldb_specs = 'BSCTRFM_{:03d}_{:03d}_07DB_MMX'.format(sldb['embedding']['hourly'], sldb['no_targets'])
sldb_specs

'BSCTRFM_168_168_07DB_MMX'

In [21]:
# get the time-based identifier for the SLDB
sldb_identifier = '{}_{}'.format(sldb['ts'], sldb_specs)
sldb_identifier

'PEMS-SF_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [22]:
sldb_dir = '/home/developer/gcp/cbidmltsf/sldbs/{}'.format(sldb_identifier)
sldb_dir

'/home/developer/gcp/cbidmltsf/sldbs/PEMS-SF_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [23]:
# get a path to the scalers sub-directory
scalers_dir = '{}/scalers'.format(sldb_dir)
scalers_dir

'/home/developer/gcp/cbidmltsf/sldbs/PEMS-SF_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX/scalers'

In [24]:
encoder_input_columns = [
    'occupancy_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week'
]

In [25]:
# both the encoder input and the decoder input use the same columns from the source sub_series dataframe
decoder_input_columns = encoder_input_columns

In [26]:
target_columns = ['occupancy_scaled']

In [27]:
id_columns = ['id']

In [28]:
# a dictionary to manage data per individual station_id
data = dict()

In [29]:
# a dictionary to store the number of examples per customer_id, stage
count = dict()

In [30]:
# the number of cores available for training in Cloud TPU
num_cores = 8

In [31]:
# are we training over raw data or preprocessed data?
state = 'raw'

In [32]:
# build a list with all the station ids

In [33]:
data_folder = '{}/PEMS-SF'.format(dataset_path)

In [34]:
def process_list(s, variable_type=int, delimiter=None):
    """Parses a line in the PEMS format to a list."""
    if delimiter is None:
      l = [
          variable_type(i) for i in s.replace('[', '').replace(']', '').split()
      ]
    else:
      l = [
          variable_type(i)
          for i in s.replace('[', '').replace(']', '').split(delimiter)
      ]

    return l

In [35]:
def read_single_list(filename):
    """Returns single list from a file in the PEMS-custom format."""
    with open(os.path.join(data_folder, filename), 'r') as dat:
        l = process_list(dat.readlines()[0])
    return l

In [36]:
station_ids = [id for id in read_single_list('stations_list')]
len(station_ids)

963

In [37]:
# from the stations list build the main data dictionary
for station_id in station_ids:
    station_data_path = '{}/separated_{}/ST_{}.pkl'.format(dataset_path, state, station_id)
    data[station_id] = pd.read_pickle(station_data_path)

In [38]:
len(data.keys())

963

In [42]:
# enumeration index starts at zero
for zero_based_sequential_id, station_id in enumerate(station_ids):
    
    # sequential_id starts at 1
    sequential_id = zero_based_sequential_id + 1

    # initialize the examples dictionary for each station
    examples = {
        'train': [],
        'eval': [],
        # test dataset is not passed to SLDB, but persisted as a time series
        # 'test': []
    }
    
    # use now a reference to the dataframe in the data dictionary
    data_df = data[station_id]
    
    print('Processing data for sensor station {}, from index {} to index {}, in the original time series.'\
          .format(station_id, data_df.index[0], data_df.index[-1])
         )
    
    # a sub-dictionary to keep the number of examples per station_id, stage
    count[station_id] = dict()
    
    # expand the dataframe with the sequential identifier
    # use a single-element list, multiplied times the number of rows in the dataframe
    data_df['sequential_id'] = [sequential_id]*data_df.shape[0]

    print('Added a column for sequential_id to the temporary dataframe for {}.'.format(station_id))

    # expand with positional encodings
    # use pd.to_numeric() to convert objects in the time series to numbers
    # this is not required for the electricity dataset!
    data_df['sin_hours_from_start'] = np.sin(2*np.pi*pd.to_numeric(data_df.hours_from_start)/total_hours)
    data_df['cos_hours_from_start'] = np.cos(2*np.pi*pd.to_numeric(data_df.hours_from_start)/total_hours)
    data_df['sin_hour_day'] = np.sin(2*np.pi*pd.to_numeric(data_df.time_on_day)/hours_in_day)
    data_df['cos_hour_day'] = np.cos(2*np.pi*pd.to_numeric(data_df.time_on_day)/hours_in_day)
    data_df['sin_day_week'] = np.sin(2*np.pi*pd.to_numeric(data_df.day_of_week)/days_in_week)
    data_df['cos_day_week'] = np.cos(2*np.pi*pd.to_numeric(data_df.day_of_week)/days_in_week)
    
    print('Added positional encodings to the temporary dataframe for {}.'.format(station_id))
    
    # get a series for the car lane occupancy variable on the training dataset, to fit the scaler
    lectures_train_data = data_df['values'].iloc[:no_lectures_train_dataset]
    
    print('Time series for training dataset for {} acquired.'.format(station_id))
    print('{} lectures from index {} to index {}.'\
          .format(lectures_train_data.index[-1] - lectures_train_data.index[0] + 1,
                  lectures_train_data.index[0],
                  lectures_train_data.index[-1]
                 )
         )

    # fit a scaler only on train data
    # it is required to pass the occupancy time series to a (?, 1) NumPy array
    lectures_train_data_array = np.array(lectures_train_data).reshape(-1, 1)
    
    # get MinMaxScaler on train data only
    scaler_type = 'min_max'
    scaler = MinMaxScaler()
    fitted_scaler = scaler.fit(lectures_train_data_array)
    print('Scaler {} generated on training data for {}'.format(scaler_type, station_id))

    # persist the scaler
    scaler_filename = '{}/{}_{}.save'.format(scalers_dir, scaler_type, station_id)
    joblib.dump(fitted_scaler, scaler_filename)
    print('Scaler {} persisted for {}'.format(scaler_type, station_id))

    # get an array from the variable time series (seen and unseen)
    # replace data_df.values with data_df['values']
    # to select only the values of the 'values' column, and not 'all-the-column' values
    all_data_variable_array = np.array(data_df['values']).reshape(-1, 1)

    # apply the scaler over all data (seen and unseen)
    # rescale, and squeeze to drop the extra dimension, then assign to the new column kw_scaled
    data_df['occupancy_scaled'] = np.squeeze(fitted_scaler.transform(all_data_variable_array))
    
    print('Added a column for occupancy_scaled to the temporary dataframe for {}.'.format(station_id))
    
    # now window-roll the individual time series to produce sub-series/examples to serialize

    # BSCTRFM inference process is not direct, but iterative, therefore
    # no TFRecord SLDB is required for test dataset

    # the time series used to build the test dataset must go
    # from lecture t3648 to lecture t4150 (starting counter from t0)
    # in order to extract 168 features with targets
    # (the last element in the decoder output)
    # ranging from lecture t3983 to lecture t4150

    test_ts_start = 3648
    test_ts_end = 4150

    # rolling over the time series for the test dataset:
    # remember the model was trained with data up to t3584, then no seen data is used for test stage
    # eval data is indeed used for test stage, but eval does not modify training processs in CloudTPU
    # first encoder input: t3648 to t3815
    # first decoder input: t3815 to t3982
    # first decoder output: t3816 to t3983,
    # then predictions from t3983 to t3983+23 (one-day-ahead) are built iteratively
    # and process is repeated for the following six days, to complete a week
    
    test_time_series = data_df[sldb_columns].iloc[test_ts_start:test_ts_end + 1]

    print('Test time series for station {} acquired: from {} to {} for {} lectures'.\
         format(station_id,
                test_time_series.index[0],
                test_time_series.index[-1],
                test_time_series.index[-1] - test_time_series.index[0] + 1))
    
    # path to persist the time series dataframe corresponding to test dataset
    path = '{}/test/ST_{}.pkl'.format(sldb_dir, station_id)

    test_time_series.to_pickle(path)
    print('Test dataset persisted as a time series pickle for {}'.format(station_id))

    
    # make SLDB training dataset
    # get an iterable with all the possible sub-series for training examples
    train_starting_indexes = np.arange(no_lectures_train_dataset - (m + t) + 1)
    
    for train_starting_index in train_starting_indexes:

        # do not substract 1 at the end of the slice because using iloc
        sub_series_df = data_df[sldb_columns].iloc[train_starting_index:train_starting_index + (m + t)]

        encoder_input_df = sub_series_df[encoder_input_columns][:m]
        decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
        target_df = sub_series_df[target_columns][m:m+t]
        id_df = sub_series_df[id_columns][:1]
        
        encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
        decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
        target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
        id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()
        
        # add a sequential_id feature to ease embedding
        sequential_id_list = [sequential_id]

        examples['train'].append(
            {
                'encoder_input': encoder_input_list,
                'decoder_input': decoder_input_list,
                'target': target_list,
                'id': id_list,
                'sequential_id': sequential_id_list
            }
        )

    print('Sensor station {} processed for a total of {} training examples.'.\
          format(station_id, len(examples['train'])))
    
    
    # make SLDB evaluation dataset
    # get an iterable with all the possible sub-series for evaluation examples
    eval_starting_indexes = np.arange(no_lectures_train_dataset,
                                      no_lectures_train_dataset + no_lectures_eval_dataset - (m + t) + 1)

    for eval_starting_index in eval_starting_indexes:

        # do not substract 1 at the end of the slice because using iloc
        sub_series_df = data_df[sldb_columns].iloc[eval_starting_index:eval_starting_index + (m + t)]

        encoder_input_df = sub_series_df[encoder_input_columns][:m]
        decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
        target_df = sub_series_df[target_columns][m:m+t]
        id_df = sub_series_df[id_columns][:1]
        
        encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
        decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
        target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
        id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()
        
        # add a sequential_id feature to ease embedding
        sequential_id_list = [sequential_id]

        examples['eval'].append(
            {
                'encoder_input': encoder_input_list,
                'decoder_input': decoder_input_list,
                'target': target_list,
                'id': id_list,
                'sequential_id': sequential_id_list
            }
        )
        
    print('Sensor station {} processed for a total of {} evaluation examples.'.\
          format(station_id, len(examples['eval'])))

    # DO NOT PRODUCE A TEST DATASET FOR SLDB, AS INFERENCE PROCESS IS NOT DIRECT
    # (IT IS ITERATIVE OVER UNSEEN DATA TIME SERIES, ALREADY PERSISTED AS A PICKLE FILE)

    # on each sensor station dataset, adjust the number of examples to the number of training cores
    for stage in ['train', 'eval']:
        # how many examples/rows must be removed from examples[stage] to comply with the number of cores
        examples_to_remove = len(examples[stage])%num_cores

        # remove the last 'examples_to_remove' examples from the dataset
        for _ in np.arange(examples_to_remove):
            examples[stage].pop(-1)

        
        # keep a record of the number of training and evaluation examples
        count[station_id][stage] = len(examples[stage])
        print('For {} cores in Cloud TPU, the number of {} examples for {} was adjusted to {}'.\
             format(num_cores, stage, station_id, len(examples[stage])))

        
    # serialize the rows in examples['train'] and, if present, examples['eval']
    # process each sensor station, then release data structures to avoid excesive memory consumption

    # write a TFRecord file for each consumer_id/stage
    for stage in ['train', 'eval']:
        # N_ROWS = sldb['stats'][stage]['n_rows']
        N_ROWS = len(examples[stage])
        filename = '{}/{}/ST_{}.tfrecord'.format(sldb_dir, stage, station_id)

        with tf.io.TFRecordWriter(filename) as writer:
            for row in np.arange(N_ROWS):

                example = tf.train.Example(
                    # features within the example
                    features=tf.train.Features(
                        # individual feature definition
                        feature={'encoder_input':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['encoder_input']),
                                 'decoder_input':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['decoder_input']),
                                 'target':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['target']),
                                 'id':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['id']),
                                 'sequential_id':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['sequential_id'])
                                 }
                    )
                )
                serialized_example = example.SerializeToString()
                writer.write(serialized_example)

            # report TFRecord file as completed
            print('Persisted {} TFRecord file for sensor station {}'.format(stage, station_id))


Processing data for sensor station 400000, from index 0 to index 4150, in the original time series.
Added a column for sequential_id to the temporary dataframe for 400000.
Added positional encodings to the temporary dataframe for 400000.
Time series for training dataset for 400000 acquired.
3585 lectures from index 0 to index 3584.
Scaler min_max generated on training data for 400000
Scaler min_max persisted for 400000
Added a column for occupancy_scaled to the temporary dataframe for 400000.
Test time series for station 400000 acquired: from 3648 to 4150 for 503 lectures
Test dataset persisted as a time series pickle for 400000
Sensor station 400000 processed for a total of 3250 training examples.
Sensor station 400000 processed for a total of 63 evaluation examples.
For 8 cores in Cloud TPU, the number of train examples for 400000 was adjusted to 3248
For 8 cores in Cloud TPU, the number of eval examples for 400000 was adjusted to 56
Persisted train TFRecord file for sensor station 4

Sensor station 400026 processed for a total of 63 evaluation examples.
For 8 cores in Cloud TPU, the number of train examples for 400026 was adjusted to 3248
For 8 cores in Cloud TPU, the number of eval examples for 400026 was adjusted to 56
Persisted train TFRecord file for sensor station 400026
Persisted eval TFRecord file for sensor station 400026
Processing data for sensor station 400027, from index 84464 to index 88614, in the original time series.
Added a column for sequential_id to the temporary dataframe for 400027.
Added positional encodings to the temporary dataframe for 400027.
Time series for training dataset for 400027 acquired.
3585 lectures from index 84464 to index 88048.
Scaler min_max generated on training data for 400027
Scaler min_max persisted for 400027
Added a column for occupancy_scaled to the temporary dataframe for 400027.
Test time series for station 400027 acquired: from 88112 to 88614 for 503 lectures
Test dataset persisted as a time series pickle for 40002

Sensor station 400041 processed for a total of 3250 training examples.
Sensor station 400041 processed for a total of 63 evaluation examples.
For 8 cores in Cloud TPU, the number of train examples for 400041 was adjusted to 3248
For 8 cores in Cloud TPU, the number of eval examples for 400041 was adjusted to 56
Persisted train TFRecord file for sensor station 400041
Persisted eval TFRecord file for sensor station 400041
Processing data for sensor station 400043, from index 168928 to index 173078, in the original time series.
Added a column for sequential_id to the temporary dataframe for 400043.
Added positional encodings to the temporary dataframe for 400043.
Time series for training dataset for 400043 acquired.
3585 lectures from index 168928 to index 172512.
Scaler min_max generated on training data for 400043
Scaler min_max persisted for 400043
Added a column for occupancy_scaled to the temporary dataframe for 400043.
Test time series for station 400043 acquired: from 172576 to 173

Sensor station 400059 processed for a total of 3250 training examples.
Sensor station 400059 processed for a total of 63 evaluation examples.
For 8 cores in Cloud TPU, the number of train examples for 400059 was adjusted to 3248
For 8 cores in Cloud TPU, the number of eval examples for 400059 was adjusted to 56
Persisted train TFRecord file for sensor station 400059
Persisted eval TFRecord file for sensor station 400059
Processing data for sensor station 400060, from index 253392 to index 257542, in the original time series.
Added a column for sequential_id to the temporary dataframe for 400060.
Added positional encodings to the temporary dataframe for 400060.
Time series for training dataset for 400060 acquired.
3585 lectures from index 253392 to index 256976.
Scaler min_max generated on training data for 400060
Scaler min_max persisted for 400060
Added a column for occupancy_scaled to the temporary dataframe for 400060.
Test time series for station 400060 acquired: from 257040 to 257

Sensor station 400078 processed for a total of 3250 training examples.
Sensor station 400078 processed for a total of 63 evaluation examples.
For 8 cores in Cloud TPU, the number of train examples for 400078 was adjusted to 3248
For 8 cores in Cloud TPU, the number of eval examples for 400078 was adjusted to 56
Persisted train TFRecord file for sensor station 400078
Persisted eval TFRecord file for sensor station 400078
Processing data for sensor station 400079, from index 337856 to index 342006, in the original time series.
Added a column for sequential_id to the temporary dataframe for 400079.
Added positional encodings to the temporary dataframe for 400079.
Time series for training dataset for 400079 acquired.
3585 lectures from index 337856 to index 341440.
Scaler min_max generated on training data for 400079
Scaler min_max persisted for 400079
Added a column for occupancy_scaled to the temporary dataframe for 400079.
Test time series for station 400079 acquired: from 341504 to 342

...

Sensor station 402018 processed for a total of 3250 training examples.
Sensor station 402018 processed for a total of 63 evaluation examples.
For 8 cores in Cloud TPU, the number of train examples for 402018 was adjusted to 3248
For 8 cores in Cloud TPU, the number of eval examples for 402018 was adjusted to 56
Persisted train TFRecord file for sensor station 402018
Persisted eval TFRecord file for sensor station 402018
Processing data for sensor station 402027, from index 9797824 to index 9801974, in the original time series.
Added a column for sequential_id to the temporary dataframe for 402027.
Added positional encodings to the temporary dataframe for 402027.
Time series for training dataset for 402027 acquired.
3585 lectures from index 9797824 to index 9801408.
Scaler min_max generated on training data for 402027
Scaler min_max persisted for 402027
Added a column for occupancy_scaled to the temporary dataframe for 402027.
Test time series for station 402027 acquired: from 9801472 t

Sensor station 402049 processed for a total of 3250 training examples.
Sensor station 402049 processed for a total of 63 evaluation examples.
For 8 cores in Cloud TPU, the number of train examples for 402049 was adjusted to 3248
For 8 cores in Cloud TPU, the number of eval examples for 402049 was adjusted to 56
Persisted train TFRecord file for sensor station 402049
Persisted eval TFRecord file for sensor station 402049
Processing data for sensor station 402050, from index 9882288 to index 9886438, in the original time series.
Added a column for sequential_id to the temporary dataframe for 402050.
Added positional encodings to the temporary dataframe for 402050.
Time series for training dataset for 402050 acquired.
3585 lectures from index 9882288 to index 9885872.
Scaler min_max generated on training data for 402050
Scaler min_max persisted for 402050
Added a column for occupancy_scaled to the temporary dataframe for 402050.
Test time series for station 402050 acquired: from 9885936 t

Sensor station 402060 processed for a total of 3250 training examples.
Sensor station 402060 processed for a total of 63 evaluation examples.
For 8 cores in Cloud TPU, the number of train examples for 402060 was adjusted to 3248
For 8 cores in Cloud TPU, the number of eval examples for 402060 was adjusted to 56
Persisted train TFRecord file for sensor station 402060
Persisted eval TFRecord file for sensor station 402060
Processing data for sensor station 402061, from index 9966752 to index 9970902, in the original time series.
Added a column for sequential_id to the temporary dataframe for 402061.
Added positional encodings to the temporary dataframe for 402061.
Time series for training dataset for 402061 acquired.
3585 lectures from index 9966752 to index 9970336.
Scaler min_max generated on training data for 402061
Scaler min_max persisted for 402061
Added a column for occupancy_scaled to the temporary dataframe for 402061.
Test time series for station 402061 acquired: from 9970400 t

Sensor station 402072 processed for a total of 3250 training examples.
Sensor station 402072 processed for a total of 63 evaluation examples.
For 8 cores in Cloud TPU, the number of train examples for 402072 was adjusted to 3248
For 8 cores in Cloud TPU, the number of eval examples for 402072 was adjusted to 56
Persisted train TFRecord file for sensor station 402072
Persisted eval TFRecord file for sensor station 402072
Processing data for sensor station 402077, from index 10051216 to index 10055366, in the original time series.
Added a column for sequential_id to the temporary dataframe for 402077.
Added positional encodings to the temporary dataframe for 402077.
Time series for training dataset for 402077 acquired.
3585 lectures from index 10051216 to index 10054800.
Scaler min_max generated on training data for 402077
Scaler min_max persisted for 402077
Added a column for occupancy_scaled to the temporary dataframe for 402077.
Test time series for station 402077 acquired: from 10054

Sensor station 402087 processed for a total of 3250 training examples.
Sensor station 402087 processed for a total of 63 evaluation examples.
For 8 cores in Cloud TPU, the number of train examples for 402087 was adjusted to 3248
For 8 cores in Cloud TPU, the number of eval examples for 402087 was adjusted to 56
Persisted train TFRecord file for sensor station 402087
Persisted eval TFRecord file for sensor station 402087
Processing data for sensor station 402088, from index 10135680 to index 10139830, in the original time series.
Added a column for sequential_id to the temporary dataframe for 402088.
Added positional encodings to the temporary dataframe for 402088.
Time series for training dataset for 402088 acquired.
3585 lectures from index 10135680 to index 10139264.
Scaler min_max generated on training data for 402088
Scaler min_max persisted for 402088
Added a column for occupancy_scaled to the temporary dataframe for 402088.
Test time series for station 402088 acquired: from 10139

In [43]:
data_df

,values,sensor_day,time_on_day,day_of_week,id,hours_from_start,sequential_id,sin_hours_from_start,cos_hours_from_start,sin_hour_day,cos_hour_day,sin_day_week,cos_day_week,occupancy_scaled
10156796,0.009600,0,1,4,402090,1,963,1.513655e-03,0.999999,0.258819,0.965926,-0.433884,-0.900969,0.015329
10156797,0.008100,0,2,4,402090,2,963,3.027307e-03,0.999995,0.500000,0.866025,-0.433884,-0.900969,0.011679
10156798,0.008817,0,3,4,402090,3,963,4.540952e-03,0.999990,0.707107,0.707107,-0.433884,-0.900969,0.013423
10156799,0.015233,0,4,4,402090,4,963,6.054586e-03,0.999982,0.866025,0.500000,-0.433884,-0.900969,0.029036
10156800,0.035267,0,5,4,402090,5,963,7.568207e-03,0.999971,0.965926,0.258819,-0.433884,-0.900969,0.077781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10160942,0.041400,172,19,5,402090,4147,963,-6.054586e-03,0.999982,-0.965926,0.258819,-0.974928,-0.222521,0.092704
10160943,0.037300,172,20,5,402090,4148,963,-4.540952e-03,0.999990,-0.866025,0.500000,-0.974928,-0.222521,0.082728
10160944,0.035400,172,21,5,402090,4149,963,-3.027307e-03,0.999995,-0.707107,0.707107,-0.974928,-0.222521,0.078105
10160945,0.029750,172,22,5,402090,4150,963,-1.513655e-03,0.999999,-0.500000,0.866025,-0.974928,-0.222521,0.064358
